In [1]:
import stac_mjx 
from pathlib import Path
import os

In [2]:
# Enable XLA flags if on GPU
stac_mjx.enable_xla_flags()

# Choose parent directory as base path for data files
base_path = Path("/root/vast/eric/stac-mjx")

# Load configs
cfg = stac_mjx.load_configs(base_path / "configs")
# cfg = stac_mjx.load_configs(base_path + "/configs")

# Load data
kp_data, sorted_kp_names = stac_mjx.load_mocap(cfg, base_path)

# Run stac
fit_path, ik_only_path = stac_mjx.run_stac(
 cfg,
 kp_data, 
 sorted_kp_names, 
 base_path=base_path
)

/root/vast/eric/stac-mjx/stac_mjx/utils.py:16: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  if xla_bridge.get_backend().platform == "gpu":


Config loaded and validated.
Running fit. Mocap data shape: (1000, 9)
Calibration iteration: 1/6
Pose Optimization:
Pose Optimization done in 92.08193325996399
Frame 1 done in 34.92336893081665 with a final error of 2.2810882072121785e-08
Frame 2 done in 0.748065710067749 with a final error of 3.4157505712784086e-09
Frame 3 done in 0.05606842041015625 with a final error of 2.535021970118123e-08
Frame 4 done in 0.05552983283996582 with a final error of 1.7416047493057363e-09
Frame 5 done in 0.0554959774017334 with a final error of 2.684633493288402e-08
Frame 6 done in 0.05539298057556152 with a final error of 1.0036386832723565e-08
Frame 7 done in 0.0554499626159668 with a final error of 1.8925586431350894e-08
Frame 8 done in 0.05539894104003906 with a final error of 1.9897930414458642e-08
Frame 9 done in 0.05533933639526367 with a final error of 1.6677405012544e-09
Frame 10 done in 0.055407047271728516 with a final error of 8.330363954200948e-09
Frame 11 done in 0.055425167083740234 wi

/root/miniforge3/envs/stac-mjx-env/lib/python3.11/site-packages/jaxopt/_src/optax_wrapper.py:120: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(update_fun, params, updates)


Final error of 0.0009573925053700805
offset optimization finished in 11.478811740875244
Calibration iteration: 2/6
Pose Optimization:
Pose Optimization done in 55.569908618927
Frame 1 done in 0.061609745025634766 with a final error of 1.1434894808814988e-08
Frame 2 done in 0.05741143226623535 with a final error of 1.6413121528557895e-08
Frame 3 done in 0.05809903144836426 with a final error of 1.828400897352367e-08
Frame 4 done in 0.05775046348571777 with a final error of 2.218118133612279e-08
Frame 5 done in 0.05809450149536133 with a final error of 2.9753765673490307e-08
Frame 6 done in 0.05759763717651367 with a final error of 9.627452612903653e-09
Frame 7 done in 0.056343793869018555 with a final error of 2.0727950911236803e-08
Frame 8 done in 0.055855512619018555 with a final error of 2.8779856720007047e-08
Frame 9 done in 0.055730581283569336 with a final error of 2.8407587393530775e-08
Frame 10 done in 0.05580449104309082 with a final error of 1.232933222183874e-08
Frame 11 done

In [ ]:
config, render = stac_mjx.viz_stac(
    data_path="/root/vast/eric/stac-mjx/mouse_arm_ik_only_new_clip_1000.h5",
    n_frames=1000,
    save_path='mouse_arm_ik_only_new_clip_1000.mp4',
    height=480,
    width=640,
)

0it [00:00, ?it/s]/root/miniforge3/envs/stac-mjx-env/lib/python3.11/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
501it [00:46, 11.54it/s]

In [ ]:
config, render = stac_mjx.viz_stac(
    data_path="/root/vast/eric/stac-mjx/mouse_arm_fit_only_new_clip_1000.h5",
    n_frames=1000,
    save_path='mouse_arm_fit_new_clip_1000.mp4',
    height=480,
    width=640,
)

In [7]:
import mujoco
import numpy as np
from tqdm import tqdm

MODEL_XML = """<?xml version="1.0" encoding="utf-8"?>
<mujoco model="mouse">
    <option timestep="0.001" gravity="0 0 0"/>
    <size njmax="1000" nconmax="400" nkey="1" nuser_jnt="1"/>
    <asset>
        <texture type="skybox" builtin="gradient" rgb1="1 1 1" rgb2="0.6 0.8 1" width="256" height="1536"/>
        <texture name="grid" type="2d" builtin="checker" width="512" height="512" rgb1=".1 .2 .3" rgb2=".2 .3 .4"/>
        <material name="grid" texture="grid" texrepeat="1 1" texuniform="true" reflectance=".2"/>
        <mesh file="/root/vast/joshua/stac-mjx/meshes/stl/Bone_Scapula_right_v3.stl"/>
        <mesh file="/root/vast/joshua/stac-mjx/meshes/stl/Bone_Humerus_right_v3.stl"/>
        <mesh file="/root/vast/joshua/stac-mjx/meshes/stl/Bone_Clavicle_right_v3.stl"/>
        <mesh file="/root/vast/joshua/stac-mjx/meshes/stl/Bone_Ulna_right_v3.stl"/>
        <mesh file="/root/vast/joshua/stac-mjx/meshes/stl/Bone_Radius_right_v3.stl"/>
        <mesh file="/root/vast/joshua/stac-mjx/meshes/stl/Bone_Segments_Hand.stl"/>
    </asset>
    <worldbody>
        <light directional="true"/>
        <body name="ground" euler="0 0 0">
            <camera name="my_camera" pos="0.04 .004 0.037" fovy="20" quat="0.6532815 0.2705981 0.2705981 0.6532815"/>
            <camera name="my_camera1" pos="0.04 .004 0.037" fovy="20" quat="0.6532815 0.2705981 0.2705981 0.6532815"/>
            <body name="clavicle" pos="0 0 0" euler="0 0 0">
                <geom type="mesh" mesh="Bone_Clavicle_right_v3" pos="0 0 0.0005"/>
                <body name="scapula" pos="-0.0001 -0.0001 0.0005" euler="-30 -15 0">
                    <geom type="mesh" mesh="Bone_Scapula_right_v3" pos="0.0 0.0 0.0"/>
                    <body name="humerus" pos="0 0.0007 -0.0002" euler="-15 0 0">
                        <inertial pos="-0.0005 0.0003 -0.004" mass="0.0001258" diaginertia="3e-8 3e-8 3e-8"/>
                        <geom type="mesh" mesh="Bone_Humerus_right_v3" pos="0.0 0.0 0.0"/>
                        <joint name="sh_elv" type="hinge" pos="0 0 0" axis="0 1 0" damping="0.001" range="-70 0" limited="true"/>
                        <joint name="sh_extension" type="hinge" pos="0 0 0" axis="0 0 1" damping="0.001" range="-70 40" limited="true"/>
                        <joint name="sh_rotation" type="hinge" pos="0 0 0" axis="1 0 0" damping="0.001" range="-25 89" limited="true"/>
                        <body name="ulna" pos="-0.0005 0.0003 -0.0068" euler="-15 0 0">
                            <geom type="mesh" mesh="Bone_Ulna_right_v3" pos="0.0 0.0 0.0"/>
                            <joint name="elbow" pos="0 0 0" axis="1 0 0" stiffness="0.000001" damping="0.00001" range="-80 90" limited="true"/>
                            <inertial pos="-0.0009 0.004 0.0005" mass="0.0001258" diaginertia="3e-8 3e-8 3e-8"/>
                            <body name="elbow" pos="0.0 0.0 0.0" euler="0 0 0">
                                <geom name="elbow_marker" type="sphere" size="0.0003" pos="0 0 0" contype="0" conaffinity="0" rgba="0.3 0.8 0.3 0.8" />
                                <body name="radius" pos="0.00035 0.0002 0.0002" euler="0 0 0">
                                    <geom type="mesh" mesh="Bone_Radius_right_v3" pos="0.0 0.0 0.0"/>
                                    <body name="hand" pos=".003 0.0005 -.009 " euler="-15 -15 -45">
                                        <geom type="mesh" mesh="Bone_Segments_Hand" pos="0 0 0"/>
                                        <inertial pos="-0.0048 0.0019 0.012" mass=" 0.00071" diaginertia="7e-08 7e-08 7e-08"/>
                                    </body>
                                </body>
                            </body>
                        </body>
                    </body>
                </body>
            </body>
        </body>
    </worldbody>
    <actuator>
        <motor name="sh_elv_act" joint="sh_elv" ctrllimited="true" ctrlrange="-1 1" forcerange="-.001 .001"/>
        <motor name="sh_extension_act" joint="sh_extension" ctrllimited="true" ctrlrange="-1 1" forcerange="-.001 .001"/>
        <motor name="sh_rotation_act" joint="sh_rotation" ctrllimited="true" ctrlrange="-1 1" forcerange="-.001 .001"/>
        <motor name="elbow_act" joint="elbow" ctrllimited="true" ctrlrange="-1 1" forcerange="-.0001 .0001"/>
    </actuator>
</mujoco>
"""

# Load and compile
model = mujoco.MjModel.from_xml_string(MODEL_XML)
data = mujoco.MjData(model)

# Forward kinematics to compute world-space positions (xpos, xquat, etc.)
mujoco.mj_forward(model, data)

for i in tqdm(range(model.nbody), desc="Bodies"):
    body_name = model.body(i).name
    parent_id = model.body_parentid[i]
    parent_name = model.body(parent_id).name if parent_id >= 0 else None

    # Local offset (relative to parent)
    local_pos = model.body_pos[i]
    local_quat = model.body_quat[i]
    
    # World coords of the body origin
    world_pos = data.xpos[i]
    world_quat = data.xquat[i]

    print(f"Body ID: {i}, Name: '{body_name}'")
    print(f"  Parent ID: {parent_id}, Parent Name: '{parent_name}'")
    print(f"  local body_pos: {local_pos}")
    print(f"  local body_quat: {local_quat}")
    print(f"  world xpos: {world_pos}")
    print(f"  world xquat: {world_quat}\n")


Bodies: 100%|██████████| 9/9 [00:00<00:00, 1237.99it/s]

Body ID: 0, Name: 'world'
  Parent ID: 0, Parent Name: 'world'
  local body_pos: [0. 0. 0.]
  local body_quat: [1. 0. 0. 0.]
  world xpos: [0. 0. 0.]
  world xquat: [1. 0. 0. 0.]

Body ID: 1, Name: 'ground'
  Parent ID: 0, Parent Name: 'world'
  local body_pos: [0. 0. 0.]
  local body_quat: [1. 0. 0. 0.]
  world xpos: [0. 0. 0.]
  world xquat: [1. 0. 0. 0.]

Body ID: 2, Name: 'clavicle'
  Parent ID: 1, Parent Name: 'ground'
  local body_pos: [0. 0. 0.]
  local body_quat: [1. 0. 0. 0.]
  world xpos: [0. 0. 0.]
  world xquat: [1. 0. 0. 0.]

Body ID: 3, Name: 'scapula'
  Parent ID: 2, Parent Name: 'clavicle'
  local body_pos: [-0.0001 -0.0001  0.0005]
  local body_quat: [ 0.9576622  -0.25660481 -0.12607862  0.03378266]
  world xpos: [-0.0001 -0.0001  0.0005]
  world xquat: [ 0.9576622  -0.25660481 -0.12607862  0.03378266]

Body ID: 4, Name: 'humerus'
  Parent ID: 3, Parent Name: 'scapula'
  local body_pos: [ 0.      0.0007 -0.0002]
  local body_quat: [ 0.99144486 -0.13052619  0.          

In [6]:
import numpy as np
import mujoco
from mujoco import mjx
import matplotlib.pyplot as plt
from pathlib import Path
from stac_mjx.stac import Stac
from stac_mjx import io, utils
import stac_mjx

# Choose parent directory as base path for data files
base_path = Path("/root/vast/eric/stac-mjx/configs")

# Load configurations
cfg = stac_mjx.load_configs(base_path)
xml_path = base_path / cfg.model.MJCF_PATH

# ✅ Step 1: Initialize STAC model
print("[INFO] Initializing STAC model...")
stac = Stac(xml_path, cfg, cfg.model.KP_NAMES)

# ✅ Step 2: Extract data directly from the STAC object
# Since we aren't loading from a file, we create mock data
print("[INFO] Extracting key model properties...")
qposes = np.zeros((1, stac._mj_model.nq))  # Single frame, zero pose
kp_data = np.zeros((1, len(cfg.model.KP_NAMES), 3))  # Zeroed keypoint positions
offsets = np.zeros((len(cfg.model.KP_NAMES), 3))  # Zero offsets

# ✅ Step 3: Set rendering parameters
n_frames = 1  # Only visualize a single frame
save_path = base_path / "stac_viz.mp4"
start_frame = 0
camera = 0
height = 480
width = 640
show_marker_error = True  # Show keypoint alignment errors

# ✅ Step 4: Render the visualization
print("[INFO] Rendering STAC model visualization...")
frames = stac.render(
    qposes=qposes,
    kp_data=kp_data,
    offsets=offsets,
    n_frames=n_frames,
    save_path=save_path,
    start_frame=start_frame,
    camera=camera,
    height=height,
    width=width,
    show_marker_error=show_marker_error,
)

print(f"[INFO] Rendering complete. Video saved at {save_path}")



Config loaded and validated.
[INFO] Initializing STAC model...
[DEBUG] Creating sites for each keypoint from config...


Creating Body Sites: 100%|██████████| 3/3 [00:00<00:00, 5737.76it/s]

[DEBUG] Attaching site 'Shoulder' to body 'scapula'
[DEBUG] - local offset from config: 0. 0. 0.

[DEBUG] Attaching site 'Elbow' to body 'elbow'
[DEBUG] - local offset from config: 0. 0. 0.

[DEBUG] Attaching site 'Wrist' to body 'wrist'
[DEBUG] - local offset from config: 0. 0. 0.


[DEBUG] Printing site positions after compilation:
[DEBUG] - Site 'Shoulder' index: 0
          local site_pos: [0. 0. 0.]
          world site_xpos: [-0.0001 -0.0001  0.0005]

[DEBUG] - Site 'Elbow' index: 1
          local site_pos: [0. 0. 0.]
          world site_xpos: [ 0.0011889  -0.00413805 -0.00495372]

[DEBUG] - Site 'Wrist' index: 2
          local site_pos: [0. 0. 0.]
          world site_xpos: [ 0.00150802 -0.00382079 -0.00495063]

[INFO] Extracting key model properties...
[INFO] Rendering STAC model visualization...



0it [00:00, ?it/s]/root/miniforge3/envs/stac-mjx-env/lib/python3.11/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
1it [00:00,  4.41it/s]

[INFO] Rendering complete. Video saved at /root/vast/eric/stac-mjx/configs/stac_viz.mp4


In [ ]:
import numpy as np
import mujoco
from mujoco import mjx
import matplotlib.pyplot as plt
from pathlib import Path
from stac_mjx.stac import Stac
from stac_mjx import io, utils
import stac_mjx
from dm_control import mjcf

# Choose parent directory as base path for data files
base_path = Path("/root/vast/eric/stac-mjx/configs")

# Load configurations
cfg = stac_mjx.load_configs(base_path)
xml_path = base_path / cfg.model.MJCF_PATH

# Step 1: Initialize STAC model
print("[INFO] Initializing STAC model...")
stac = Stac(xml_path, cfg, cfg.model.KP_NAMES)

# Step 2: Extract key model properties
print("[INFO] Extracting key model properties...")
qposes = np.zeros((1, stac._mj_model.nq))  # Single frame, zero pose
kp_data = np.zeros((1, len(cfg.model.KP_NAMES), 3))  # Zeroed keypoint positions
offsets = np.zeros((len(cfg.model.KP_NAMES), 3))  # Zero offsets

# Step 3: Add explicit body sites as sphere markers
print("[INFO] Adding keypoint markers as sphere sites in MJCF model...")

for key, body_name in cfg.model.KEYPOINT_MODEL_PAIRS.items():
    parent = stac._root.find("body", body_name)
    if parent is None:
        print(f"[WARNING] Could not find body '{body_name}' for keypoint '{key}', skipping...")
        continue
    
    pos_array = cfg.model.KEYPOINT_INITIAL_OFFSETS[key]  # Directly extract float list

    # Add a new site for visualization
    parent.add(
        "site",
        name=f"{key}_marker",
        type="sphere",
        size=[0.002],  # Bigger for visibility
        rgba="0 0 0 .5",  # Red color for markers
        pos=pos_array,
        group=3,
    )

# Step 4: Recompile the MuJoCo model after adding sites
print("[INFO] Compiling the updated MJCF model with site markers...")
physics = mjcf.Physics.from_mjcf_model(stac._root)

# Debug Output: Print Model XML to Verify Sites Exist
#print("[DEBUG] Updated MJCF Model:\n")
#print(stac._root.to_xml_string())  # 👀 Check if sites are correctly added

# Ensure `stac._mj_model` gets the updated compiled model
stac._mj_model = physics.model.ptr

# Step 5: Set rendering parameters
n_frames = 1  # Only visualize a single frame
save_path = base_path / "stac_viz.mp4"
start_frame = 0
camera = 0
height = 480
width = 640
show_marker_error = True  # Show keypoint alignment errors

# Step 6: Render the visualization
print("[INFO] Rendering STAC model visualization with site markers...")
frames = stac.render(
    qposes=qposes,
    kp_data=kp_data,
    offsets=offsets,  # Ensure correct offset positions
    n_frames=n_frames,
    save_path=save_path,
    start_frame=start_frame,
    camera=camera,
    height=height,
    width=width,
    show_marker_error=True,  # 🔥 This will now highlight marker alignment errors
)

print(f"[INFO] Rendering complete. Video saved at {save_path}")

Config loaded and validated.
[INFO] Initializing STAC model...
[DEBUG] Creating sites for each keypoint from config...


Creating Body Sites: 100%|██████████| 3/3 [00:00<00:00, 5688.48it/s]

[DEBUG] Attaching site 'Shoulder' to body 'scapula'
[DEBUG] Attaching site 'Elbow' to body 'elbow'
[DEBUG] Attaching site 'Wrist' to body 'hand'

[DEBUG] Printing site positions after compilation:
[DEBUG] - Site 'Shoulder' index: 0
          local site_pos: [0. 0. 0.]
          world site_xpos: [-0.0001 -0.0001  0.0005]

[DEBUG] - Site 'Elbow' index: 1
          local site_pos: [0. 0. 0.]
          world site_xpos: [ 0.0011889  -0.00413805 -0.00495372]

[DEBUG] - Site 'Wrist' index: 2
          local site_pos: [0. 0. 0.]
          world site_xpos: [ 0.0064878  -0.01083974 -0.00897384]



[INFO] Extracting key model properties...
[INFO] Adding keypoint markers as sphere sites in MJCF model...
[INFO] Compiling the updated MJCF model with site markers...
[INFO] Rendering STAC model visualization with site markers...


0it [00:00, ?it/s]/root/miniforge3/envs/stac-mjx-env/lib/python3.11/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
1it [00:00,  4.07it/s]

[INFO] Rendering complete. Video saved at /root/vast/eric/stac-mjx/configs/stac_viz.mp4


In [3]:
import numpy as np
import mujoco
from mujoco import mjx
import matplotlib.pyplot as plt
from pathlib import Path
from stac_mjx.stac import Stac
from stac_mjx import io, utils
import stac_mjx
from dm_control import mjcf

# Choose parent directory as base path for data files
base_path = Path("/root/vast/eric/stac-mjx/configs")

# Load configurations
cfg = stac_mjx.load_configs(base_path)
xml_path = base_path / cfg.model.MJCF_PATH

# Step 1: Initialize STAC model
print("[INFO] Initializing STAC model...")
stac = Stac(xml_path, cfg, cfg.model.KP_NAMES)

# Step 2: Extract key model properties
print("[INFO] Extracting key model properties...")
n_frames = 1  # Visualize 1 frame
qposes = np.zeros((n_frames, stac._mj_model.nq))  # Default pose
kp_data = np.zeros((n_frames, len(cfg.model.KP_NAMES), 3))  # Zeroed keypoint positions
offsets = np.zeros((len(cfg.model.KP_NAMES), 3))  # Zero offsets

# Step 3: Attach keypoint markers to MJCF Model BEFORE recompilation
print("[INFO] Adding keypoint markers as sphere sites in MJCF model...")

for key, body_name in cfg.model.KEYPOINT_MODEL_PAIRS.items():
    parent = stac._root.find("body", body_name)
    if parent is None:
        print(f"[WARNING] Could not find body '{body_name}' for keypoint '{key}', skipping...")
        continue

    pos_array = cfg.model.KEYPOINT_INITIAL_OFFSETS[key]

    # Add a new site for visualization
    parent.add(
        "site",
        name=f"{key}_marker",
        type="sphere",
        size=[0.002],  # Bigger for visibility
        rgba="0 0 0 .4",  # Red color for visibility
        pos=pos_array,
        group=3,
    )

# Step 4: Recompile MJCF model to apply modifications
print("[INFO] Compiling the updated MJCF model with site markers...")
physics = mjcf.Physics.from_mjcf_model(stac._root)

# Step 5: Update `stac._mj_model` after applying changes
stac._mj_model = physics.model.ptr

# Step 6: Set rendering parameters
save_path = base_path / "stac_viz_fixed.mp4"
start_frame = 0
camera = 0
height = 480
width = 640
show_marker_error = True

# Step 7: Render the visualization with the FIXED skeleton
print("[INFO] Rendering STAC model visualization with site markers...")
frames = stac.render(
    qposes=qposes,
    kp_data=kp_data,
    offsets=offsets,
    n_frames=n_frames,
    save_path=save_path,
    start_frame=start_frame,
    camera=camera,
    height=height,
    width=width,
    show_marker_error=True,  # 🔥 Will highlight marker alignment issues
)

print(f"[INFO] Rendering complete. Video saved at {save_path}")


Config loaded and validated.
[INFO] Initializing STAC model...
[DEBUG] Creating sites for each keypoint from config...


Creating Body Sites: 100%|██████████| 3/3 [00:00<00:00, 5971.96it/s]

[DEBUG] Attaching site 'Shoulder' to body 'scapula'
[DEBUG] Attaching site 'Elbow' to body 'elbow'
[DEBUG] Attaching site 'Wrist' to body 'wrist'

[DEBUG] Printing site positions after compilation:
[DEBUG] - Site 'Shoulder' index: 0
          local site_pos: [0. 0. 0.]
          world site_xpos: [-0.0001 -0.0001  0.0005]

[DEBUG] - Site 'Elbow' index: 1
          local site_pos: [0. 0. 0.]
          world site_xpos: [ 0.0011889  -0.00413805 -0.00495372]

[DEBUG] - Site 'Wrist' index: 2
          local site_pos: [0. 0. 0.]
          world site_xpos: [ 0.00030526 -0.00034861 -0.01069496]

[INFO] Extracting key model properties...
[INFO] Adding keypoint markers as sphere sites in MJCF model...
[INFO] Compiling the updated MJCF model with site markers...


[INFO] Rendering STAC model visualization with site markers...


1it [00:00,  5.30it/s]


[INFO] Rendering complete. Video saved at /root/vast/eric/stac-mjx/configs/stac_viz_fixed.mp4
